## Step 0

In [ ]:
# ЛОГИЧЕСКИЙ БЛОК: imports + reproducibility + GLOBAL config
# ИСПОЛНЕНИЕ БЛОКА:

import os, math, random
import numpy as np
import pandas as pd
from pathlib import Path

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from sklearn.preprocessing import RobustScaler
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, roc_auc_score

def seed_everything(seed=1234):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(100)

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("DEVICE:", DEVICE)

# -------------------------------
# GLOBAL CONFIG (всё тут)
# -------------------------------
CFG = {
    # data
    "freq": "1min",
    "data_dir": Path("../dataset"),

    "lookback": 96,
    "tb_horizon": 24,          
    "book_levels": 15,         # сколько уровней стакана грузим
    "top_levels": 5,           # DI_L0..DI_L4
    "near_levels": 5,          # near=0..4, far=5..14

    # walk-forward windows (в sample-space)
    "train_min_frac": 0.50,
    "val_window_frac": 0.10,
    "test_window_frac": 0.10,
    "step_window_frac": 0.10,

    # scaling
    "max_abs_feat": 10.0,

    # correlations
    "corr_windows": [6, 12, 24, 48, 96],  # 30m,1h,2h,4h,8h
    "edges": [("ADA","BTC"), ("ADA","ETH"), ("ETH","BTC")],

    # triple-barrier (labels)
    "tb_horizon": 4*60,       # 4h     # нужен для sample_t (чтобы TB-exit не вылезал за конец)
    "tb_vol_window": 8*60,    # 8h
    "tb_pt_mult": 3,
    "tb_sl_mult": 3,
    "tb_min_barrier": 0.001,
    "tb_max_barrier": 0.006,
    # training (общие)
    "batch_size": 256,
    "epochs": 30,
    "lr": 2e-4,
    "weight_decay": 1e-3,
    "grad_clip": 1.0,
    "dropout": 0.15,
    "hidden": 64,
    "gnn_layers": 2,
    "lstm_hidden": 64,
    "lstm_layers": 1,
    "use_amp": True,

    # trading eval
    "cost_bps": 2.0,

    # confidence thresholds (для PnL по порогу)
    "thr_trade_grid": [0.50, 0.55, 0.60, 0.65, 0.70],
    "thr_dir_grid":   [0.50, 0.55, 0.60, 0.65, 0.70],
}

ASSETS = ["ADA", "BTC", "ETH"]
ASSET2IDX = {a:i for i,a in enumerate(ASSETS)}
TARGET_ASSET = "ETH"
TARGET_NODE = ASSET2IDX[TARGET_ASSET]

EDGES = CFG["edges"]
EDGE_INDEX = torch.tensor([[ASSET2IDX[s], ASSET2IDX[t]] for (s,t) in EDGES], dtype=torch.long)  # [E,2]
print("EDGE_INDEX:", EDGE_INDEX.tolist())


DEVICE: cpu
EDGE_INDEX: [[0, 1], [0, 2], [2, 1]]


## 1. load data + basic returns

In [2]:
# ЛОГИЧЕСКИЙ БЛОК: load data + log returns (без target) + все уровни стакана
# ИСПОЛНЕНИЕ БЛОКА:

def load_asset(asset: str, freq: str, data_dir: Path, book_levels: int) -> pd.DataFrame:
    path = data_dir / f"{asset}_{freq}.csv"
    df = pd.read_csv(path)
    df["timestamp"] = pd.to_datetime(df["system_time"]).dt.round("min")
    df = df.sort_values("timestamp").set_index("timestamp")

    bid_cols = [f"bids_notional_{i}" for i in range(book_levels)]
    ask_cols = [f"asks_notional_{i}" for i in range(book_levels)]

    needed = ["midpoint", "spread", "buys", "sells"] + bid_cols + ask_cols
    missing = [c for c in needed if c not in df.columns]
    if missing:
        raise ValueError(f"{asset}: missing columns in CSV: {missing[:10]}{'...' if len(missing) > 10 else ''}")

    return df[needed]


def load_all_assets() -> pd.DataFrame:
    freq = CFG["freq"]
    data_dir = CFG["data_dir"]
    book_levels = CFG["book_levels"]

    def rename_asset_cols(df_one: pd.DataFrame, asset: str) -> pd.DataFrame:
        rename_map = {
            "midpoint": asset,
            "buys": f"buys_{asset}",
            "sells": f"sells_{asset}",
            "spread": f"spread_{asset}",
        }
        for i in range(book_levels):
            rename_map[f"bids_notional_{i}"] = f"bids_vol_{asset}_{i}"
            rename_map[f"asks_notional_{i}"] = f"asks_vol_{asset}_{i}"
        return df_one.rename(columns=rename_map)

    df_ADA = rename_asset_cols(load_asset("ADA", freq, data_dir, book_levels), "ADA")
    df_BTC = rename_asset_cols(load_asset("BTC", freq, data_dir, book_levels), "BTC")
    df_ETH = rename_asset_cols(load_asset("ETH", freq, data_dir, book_levels), "ETH")

    df = df_ADA.join(df_BTC).join(df_ETH)
    df = df.reset_index()  # timestamp column remains
    return df


df = load_all_assets()
T = len(df)

# log returns
for a in ASSETS:
    df[f"lr_{a}"] = np.log(df[a]).diff().fillna(0.0)

print("Loaded df:", df.shape)
print("Example columns:", df.columns[:25].tolist())


Loaded df: (17109, 106)
Example columns: ['timestamp', 'ADA', 'spread_ADA', 'buys_ADA', 'sells_ADA', 'bids_vol_ADA_0', 'bids_vol_ADA_1', 'bids_vol_ADA_2', 'bids_vol_ADA_3', 'bids_vol_ADA_4', 'bids_vol_ADA_5', 'bids_vol_ADA_6', 'bids_vol_ADA_7', 'bids_vol_ADA_8', 'bids_vol_ADA_9', 'bids_vol_ADA_10', 'bids_vol_ADA_11', 'bids_vol_ADA_12', 'bids_vol_ADA_13', 'bids_vol_ADA_14', 'asks_vol_ADA_0', 'asks_vol_ADA_1', 'asks_vol_ADA_2', 'asks_vol_ADA_3', 'asks_vol_ADA_4']


## 2. multi-window correlations → edge features (T,E,W)

In [3]:
# ЛОГИЧЕСКИЙ БЛОК: multi-window correlations -> corr_array (T,E,W)
# ИСПОЛНЕНИЕ БЛОКА:

candidate_windows = CFG["corr_windows"]
edges = EDGES

n_w = len(candidate_windows)
n_edges = len(edges)
T = len(df)

corr_array = np.zeros((T, n_edges, n_w), dtype=np.float32)

for wi, w in enumerate(candidate_windows):
    r_ADA_BTC = df["lr_ADA"].rolling(w, min_periods=1).corr(df["lr_BTC"])
    r_ADA_ETH = df["lr_ADA"].rolling(w, min_periods=1).corr(df["lr_ETH"])
    r_ETH_BTC = df["lr_ETH"].rolling(w, min_periods=1).corr(df["lr_BTC"])

    corr_array[:, 0, wi] = np.nan_to_num(r_ADA_BTC)
    corr_array[:, 1, wi] = np.nan_to_num(r_ADA_ETH)
    corr_array[:, 2, wi] = np.nan_to_num(r_ETH_BTC)

print("corr_array shape:", corr_array.shape)  # (T,E,W)


corr_array shape: (17109, 3, 5)


## 3. triple-barrier → y_tb + exit_ret → two-stage labels

In [4]:
# ЛОГИЧЕСКИЙ БЛОК: triple-barrier labels -> y_tb + exit_ret + two-stage labels
# ИСПОЛНЕНИЕ БЛОКА:

def triple_barrier_labels_from_lr(
    lr: pd.Series,
    horizon: int,
    vol_window: int,
    pt_mult: float,
    sl_mult: float,
    min_barrier: float,
    max_barrier: float,
):
    """
    Returns:
      y_tb: {0=down, 1=flat/no-trade, 2=up}
      exit_ret: realized log-return to exit (tp/sl/timeout)
      exit_t: exit index
      thr: barrier per t
    No leakage: vol is shift(1).
    """
    lr = lr.astype(float).copy()
    T = len(lr)

    vol = lr.rolling(vol_window, min_periods=max(10, vol_window//10)).std().shift(1)
    thr = (vol * np.sqrt(horizon)).clip(lower=min_barrier, upper=max_barrier)

    y = np.ones(T, dtype=np.int64)
    exit_ret = np.zeros(T, dtype=np.float32)
    exit_t = np.arange(T, dtype=np.int64)

    lr_np = lr.fillna(0.0).to_numpy(dtype=np.float64)
    thr_np = thr.fillna(min_barrier).to_numpy(dtype=np.float64)

    for t in range(T - horizon - 1):
        up = pt_mult * thr_np[t]
        dn = -sl_mult * thr_np[t]

        cum = 0.0
        hit = 1
        et = t + horizon
        er = 0.0

        for dt in range(1, horizon + 1):
            cum += lr_np[t + dt]
            if cum >= up:
                hit = 2
                et = t + dt
                er = cum
                break
            if cum <= dn:
                hit = 0
                et = t + dt
                er = cum
                break

        if hit == 1:
            er = float(np.sum(lr_np[t+1:t+horizon+1]))
            et = t + horizon

        y[t] = hit
        exit_ret[t] = er
        exit_t[t] = et

    return y, exit_ret, exit_t, thr_np

# --- build TB on ETH ---
y_tb, exit_ret, exit_t, thr = triple_barrier_labels_from_lr(
    df["lr_ETH"],
    horizon=4*60, 
    vol_window=8*60,
    pt_mult=3,
    sl_mult=3,
    min_barrier=0.001,
    max_barrier=0.006,
)

# two-stage labels
y_trade = (y_tb != 1).astype(np.int64)      # 1=trade, 0=no-trade
y_dir   = (y_tb == 2).astype(np.int64)      # 1=up, 0=down (для trade-сэмплов)

print("TB dist [down,flat,up]:", np.bincount(y_tb, minlength=3))
print("Trade ratio:", y_trade.mean())


TB dist [down,flat,up]: [3996 9005 4108]
Trade ratio: 0.47366882927114384


## 4. build node tensor + edge tensor + sample_t

In [5]:
# ЛОГИЧЕСКИЙ БЛОК: build node features (T,N,F) + edge features (T,E,W) + sample_t
# ИСПОЛНЕНИЕ БЛОКА:

EPS = 1e-6

def safe_log1p(x: np.ndarray) -> np.ndarray:
    return np.log1p(np.maximum(x, 0.0))

def build_node_tensor(df: pd.DataFrame):
    """
    Фичи на asset:
      lr, spread,
      log_buys, log_sells, ofi,
      DI_15,
      DI_L0..DI_L4,
      near_ratio_bid, near_ratio_ask,
      di_near, di_far
    """
    feats = []
    feat_names = [
        "lr", "spread",
        "log_buys", "log_sells", "ofi",
        "DI_15",
        "DI_L0", "DI_L1", "DI_L2", "DI_L3", "DI_L4",
        "near_ratio_bid", "near_ratio_ask",
        "di_near", "di_far",
    ]

    book_levels = CFG["book_levels"]
    top_k = CFG["top_levels"]     # 5
    near_k = CFG["near_levels"]   # 5
    far_k = book_levels - near_k
    if far_k <= 0:
        raise ValueError("CFG['near_levels'] must be < CFG['book_levels']")

    for a in ASSETS:
        lr = df[f"lr_{a}"].values.astype(np.float32)
        spread = df[f"spread_{a}"].values.astype(np.float32)

        buys = df[f"buys_{a}"].values.astype(np.float32)
        sells = df[f"sells_{a}"].values.astype(np.float32)

        log_buys = safe_log1p(buys).astype(np.float32)
        log_sells = safe_log1p(sells).astype(np.float32)

        ofi = ((buys - sells) / (buys + sells + EPS)).astype(np.float32)

        # уровни стакана
        bids_lvls = np.stack([df[f"bids_vol_{a}_{i}"].values.astype(np.float32) for i in range(book_levels)], axis=1)  # (T,15)
        asks_lvls = np.stack([df[f"asks_vol_{a}_{i}"].values.astype(np.float32) for i in range(book_levels)], axis=1)  # (T,15)

        bid_sum_15 = bids_lvls.sum(axis=1)
        ask_sum_15 = asks_lvls.sum(axis=1)
        DI_15 = ((bid_sum_15 - ask_sum_15) / (bid_sum_15 + ask_sum_15 + EPS)).astype(np.float32)

        # DI_L0..DI_L4
        di_levels = []
        for i in range(top_k):
            b = bids_lvls[:, i]
            s = asks_lvls[:, i]
            di_levels.append(((b - s) / (b + s + EPS)).astype(np.float32))
        DI_L0_4 = np.stack(di_levels, axis=1)  # (T,5)

        # near vs far
        bid_near = bids_lvls[:, :near_k].sum(axis=1)
        ask_near = asks_lvls[:, :near_k].sum(axis=1)
        bid_far = bids_lvls[:, near_k:].sum(axis=1)
        ask_far = asks_lvls[:, near_k:].sum(axis=1)

        near_ratio_bid = (bid_near / (bid_far + EPS)).astype(np.float32)
        near_ratio_ask = (ask_near / (ask_far + EPS)).astype(np.float32)

        di_near = ((bid_near - ask_near) / (bid_near + ask_near + EPS)).astype(np.float32)
        di_far = ((bid_far - ask_far) / (bid_far + ask_far + EPS)).astype(np.float32)

        Xa = np.column_stack([
            lr, spread,
            log_buys, log_sells, ofi,
            DI_15,
            DI_L0_4[:, 0], DI_L0_4[:, 1], DI_L0_4[:, 2], DI_L0_4[:, 3], DI_L0_4[:, 4],
            near_ratio_bid, near_ratio_ask,
            di_near, di_far
        ]).astype(np.float32)

        feats.append(Xa)

    X = np.stack(feats, axis=1).astype(np.float32)  # (T,N,F)
    return X, feat_names


X_node_raw, node_feat_names = build_node_tensor(df)
edge_feat = np.nan_to_num(corr_array.astype(np.float32), nan=0.0, posinf=0.0, neginf=0.0)

T = len(df)
L = CFG["lookback"]
H = CFG["tb_horizon"]

# sample_t: чтобы можно было брать окно [t-L+1 ... t] и иметь будущий TB-exit без выхода за данные
t_min = L - 1
t_max = T - H - 2
sample_t = np.arange(t_min, t_max + 1)
n_samples = len(sample_t)

print("X_node_raw:", X_node_raw.shape, "edge_feat:", edge_feat.shape)
print("node_feat_names:", node_feat_names)
print("n_samples:", n_samples, "t range:", sample_t[0], sample_t[-1])


X_node_raw: (17109, 3, 15) edge_feat: (17109, 3, 5)
node_feat_names: ['lr', 'spread', 'log_buys', 'log_sells', 'ofi', 'DI_15', 'DI_L0', 'DI_L1', 'DI_L2', 'DI_L3', 'DI_L4', 'near_ratio_bid', 'near_ratio_ask', 'di_near', 'di_far']
n_samples: 16773 t range: 95 16867



## 5. walk-forward splits (с глобальными окнами)

In [6]:
# ЛОГИЧЕСКИЙ БЛОК: walk-forward splits (expanding train + fixed val/test)
# ИСПОЛНЕНИЕ БЛОКА:

def make_walk_forward_splits(n_samples: int,
                             train_min_frac: float,
                             val_window_frac: float,
                             test_window_frac: float,
                             step_window_frac: float):
    train_min = int(train_min_frac * n_samples)
    val_w  = max(1, int(val_window_frac * n_samples))
    test_w = max(1, int(test_window_frac * n_samples))
    step_w = max(1, int(step_window_frac * n_samples))

    splits = []
    start = train_min
    while True:
        tr_end = start
        va_end = tr_end + val_w
        te_end = va_end + test_w
        if te_end > n_samples:
            break
        idx_train = np.arange(0, tr_end)
        idx_val   = np.arange(tr_end, va_end)
        idx_test  = np.arange(va_end, te_end)
        splits.append((idx_train, idx_val, idx_test))
        start += step_w
    return splits

walk_splits = make_walk_forward_splits(
    n_samples=n_samples,
    train_min_frac=CFG["train_min_frac"],
    val_window_frac=CFG["val_window_frac"],
    test_window_frac=CFG["test_window_frac"],
    step_window_frac=CFG["step_window_frac"],
)

print("n_folds:", len(walk_splits))

for i, (a,b,c) in enumerate(walk_splits):
    print(f" fold {i+1}: train {len(a)} | val {len(b)} | test {len(c)}")


n_folds: 4
 fold 1: train 8386 | val 1677 | test 1677
 fold 2: train 10063 | val 1677 | test 1677
 fold 3: train 11740 | val 1677 | test 1677
 fold 4: train 13417 | val 1677 | test 1677


## 6. Dataset + scaling 

In [7]:
# ЛОГИЧЕСКИЙ БЛОК: Dataset + scaling (shared)
# ИСПОЛНЕНИЕ БЛОКА:

class LobGraphSequenceDataset2Stage(Dataset):
    """
    Возвращает (x_seq, e_seq, y_trade, y_dir, exit_ret)
    y_dir корректен только когда y_trade==1, но мы возвращаем всегда.
    """
    def __init__(self, X_node, E_feat, y_trade, y_dir, exit_ret, sample_t, indices, lookback):
        self.X_node = X_node
        self.E_feat = E_feat
        self.y_trade = y_trade
        self.y_dir = y_dir
        self.exit_ret = exit_ret
        self.sample_t = sample_t
        self.indices = indices
        self.L = lookback

    def __len__(self):
        return len(self.indices)

    def __getitem__(self, i):
        sidx = self.indices[i]
        t = self.sample_t[sidx]
        t0 = t - self.L + 1

        x_seq = self.X_node[t0:t+1]     # (L,N,F)
        e_seq = self.E_feat[t0:t+1]     # (L,E,W)

        yt = self.y_trade[t]
        yd = self.y_dir[t]
        er = self.exit_ret[t]

        return (
            torch.from_numpy(x_seq),
            torch.from_numpy(e_seq),
            torch.tensor(yt, dtype=torch.long),
            torch.tensor(yd, dtype=torch.long),
            torch.tensor(er, dtype=torch.float32),
        )

def collate_fn_2stage(batch):
    xs, es, yts, yds, ers = zip(*batch)
    return (
        torch.stack(xs, 0),   # (B,L,N,F)
        torch.stack(es, 0),   # (B,L,E,W)
        torch.stack(yts, 0),  # (B,)
        torch.stack(yds, 0),  # (B,)
        torch.stack(ers, 0),  # (B,)
    )

def fit_scale_nodes_train_only(X_node_raw, sample_t, idx_train, max_abs=10.0):
    """
    Fit scaler on all times up to last train sample time (без leakage).
    """
    last_train_t = sample_t[idx_train[-1]]
    train_time_mask = np.arange(0, last_train_t + 1)

    X_train_time = X_node_raw[train_time_mask]  # (Ttr,N,F)
    Ttr, N, Fdim = X_train_time.shape

    scaler = RobustScaler(with_centering=True, with_scaling=True, quantile_range=(5.0, 95.0))
    scaler.fit(X_train_time.reshape(-1, Fdim))

    X_scaled = scaler.transform(X_node_raw.reshape(-1, Fdim)).reshape(X_node_raw.shape).astype(np.float32)
    X_scaled = np.clip(X_scaled, -max_abs, max_abs).astype(np.float32)
    X_scaled = np.nan_to_num(X_scaled, nan=0.0, posinf=0.0, neginf=0.0).astype(np.float32)
    return X_scaled, scaler

def subset_trade_indices(indices, sample_t, y_trade):
    """
    indices в sample-space -> отфильтровать те, где y_trade[t]==1
    """
    tt = sample_t[indices]
    mask = (y_trade[tt] == 1)
    return indices[mask]


## 7.Model (один класс, n_classes=2) + EdgeGatedMP

In [8]:
# ЛОГИЧЕСКИЙ БЛОК: GNN + LSTM classifier (универсальный под 2 класса)
# ИСПОЛНЕНИЕ БЛОКА:

class EdgeGatedMP(nn.Module):
    def __init__(self, in_dim, hidden, edge_dim, dropout=0.1):
        super().__init__()
        self.node_proj = nn.Linear(in_dim, hidden)
        self.ln0 = nn.LayerNorm(hidden)

        self.edge_mlp = nn.Sequential(
            nn.Linear(2*hidden + edge_dim, 2*hidden),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(2*hidden, hidden + 1)  # msg(hidden) + gate(1)
        )

        self.upd = nn.Sequential(
            nn.Linear(2*hidden, 2*hidden),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(2*hidden, hidden)
        )
        self.ln1 = nn.LayerNorm(hidden)
        self.dropout = nn.Dropout(dropout)

    def forward_once(self, x_t, edge_attr_t, edge_index):
        B, N, _ = x_t.shape
        E = edge_index.shape[0]

        h = self.ln0(self.node_proj(x_t))  # (B,N,H)
        h = torch.nan_to_num(h, nan=0.0, posinf=0.0, neginf=0.0)

        agg = torch.zeros((B, N, h.shape[-1]), device=h.device, dtype=h.dtype)

        for e in range(E):
            src = edge_index[e, 0].item()
            dst = edge_index[e, 1].item()
            h_src = h[:, src, :]
            h_dst = h[:, dst, :]
            ea = edge_attr_t[:, e, :]

            z = torch.cat([h_src, h_dst, ea], dim=-1)
            out = self.edge_mlp(z)
            msg = out[:, :-1]
            gate = torch.sigmoid(out[:, -1:])

            agg[:, dst, :] += msg * gate

        h2 = self.upd(torch.cat([h, agg], dim=-1))
        h2 = self.ln1(h + self.dropout(h2))
        h2 = torch.nan_to_num(h2, nan=0.0, posinf=0.0, neginf=0.0)
        return h2

    def forward(self, x_seq, e_seq, edge_index):
        B, L, N, Fin = x_seq.shape
        h_out = []
        for t in range(L):
            ht = self.forward_once(x_seq[:, t, :, :], e_seq[:, t, :, :], edge_index)
            h_out.append(ht)
        return torch.stack(h_out, dim=1)  # (B,L,N,H)

class GNN_LSTM_Classifier(nn.Module):
    def __init__(self, node_in, edge_dim, hidden, gnn_layers, lstm_hidden, lstm_layers,
                 dropout=0.1, target_node=2, n_classes=2):
        super().__init__()
        self.target_node = target_node

        self.gnns = nn.ModuleList()
        for i in range(gnn_layers):
            in_dim = node_in if i == 0 else hidden
            self.gnns.append(EdgeGatedMP(in_dim=in_dim, hidden=hidden, edge_dim=edge_dim, dropout=dropout))

        self.lstm = nn.LSTM(
            input_size=hidden,
            hidden_size=lstm_hidden,
            num_layers=lstm_layers,
            batch_first=True,
            dropout=dropout if lstm_layers > 1 else 0.0
        )

        self.head = nn.Sequential(
            nn.LayerNorm(lstm_hidden),
            nn.Dropout(dropout),
            nn.Linear(lstm_hidden, lstm_hidden),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(lstm_hidden, n_classes)
        )

        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)

    def forward(self, x, e, edge_index):
        h = x
        for gnn in self.gnns:
            h = gnn(h, e, edge_index)  # (B,L,N,H)

        h_tgt = h[:, :, self.target_node, :]  # (B,L,H)
        out, _ = self.lstm(h_tgt)
        last = out[:, -1, :]
        logits = self.head(last)
        return torch.nan_to_num(logits, nan=0.0, posinf=0.0, neginf=0.0)

print("Model ready.")


Model ready.


## 8. Training/Eval: Stage A (trade) и Stage B (direction)

In [9]:
# ЛОГИЧЕСКИЙ БЛОК: train/eval helpers for two-stage
# ИСПОЛНЕНИЕ БЛОКА:

@torch.no_grad()
def eval_binary(model, loader, loss_fn):
    model.eval()
    ys = []
    probs = []
    ers = []
    total_loss = 0.0
    n = 0

    for x, e, y_trade_b, y_dir_b, er in loader:
        x = x.to(DEVICE).float()
        e = e.to(DEVICE).float()
        er_np = er.cpu().numpy()

        logits = model(x, e, EDGE_INDEX.to(DEVICE))  # (B,2)
        # y is passed inside loader by selecting correct field externally
        # so here we assume loader yields y in y_trade_b (or y_dir_b) as "y_trade_b"
        y = y_trade_b.to(DEVICE).long()

        loss = loss_fn(logits, y)
        total_loss += loss.item() * y.size(0)
        n += y.size(0)

        p = torch.softmax(logits, dim=-1).detach().cpu().numpy()  # (B,2)
        ys.append(y.detach().cpu().numpy())
        probs.append(p)
        ers.append(er_np)

    ys = np.concatenate(ys)
    probs = np.concatenate(probs)
    ers = np.concatenate(ers)

    y_pred = probs.argmax(axis=1)
    acc = accuracy_score(ys, y_pred)
    f1m = f1_score(ys, y_pred, average="macro")
    auc = roc_auc_score(ys, probs[:,1]) if len(np.unique(ys)) == 2 else np.nan
    cm = confusion_matrix(ys, y_pred)

    return total_loss/max(n,1), acc, f1m, auc, cm, ys, probs, ers

def make_pos_weight(y01: np.ndarray):
    pos = max(1, int((y01 == 1).sum()))
    neg = max(1, int((y01 == 0).sum()))
    return float(neg / pos)

def train_binary_classifier(
    X_scaled, edge_feat,
    y_trade_arr, y_dir_arr,
    exit_ret, sample_t,
    idx_train, idx_val, idx_test,
    cfg,
    stage_name: str,
):
    """
    stage_name:
      - "trade": обучаем y_trade на всех samples
      - "dir":   обучаем y_dir только на trade samples (idx_* уже должны быть отфильтрованы)
    """
    L = cfg["lookback"]

    tr_ds = LobGraphSequenceDataset2Stage(X_scaled, edge_feat, y_trade_arr, y_dir_arr, exit_ret, sample_t, idx_train, L)
    va_ds = LobGraphSequenceDataset2Stage(X_scaled, edge_feat, y_trade_arr, y_dir_arr, exit_ret, sample_t, idx_val, L)
    te_ds = LobGraphSequenceDataset2Stage(X_scaled, edge_feat, y_trade_arr, y_dir_arr, exit_ret, sample_t, idx_test, L)

    tr_loader = DataLoader(tr_ds, batch_size=cfg["batch_size"], shuffle=True, drop_last=True, collate_fn=collate_fn_2stage)
    va_loader = DataLoader(va_ds, batch_size=cfg["batch_size"], shuffle=False, collate_fn=collate_fn_2stage)
    te_loader = DataLoader(te_ds, batch_size=cfg["batch_size"], shuffle=False, collate_fn=collate_fn_2stage)

    node_in = X_scaled.shape[-1]
    edge_dim = edge_feat.shape[-1]
    model = GNN_LSTM_Classifier(
        node_in=node_in, edge_dim=edge_dim,
        hidden=cfg["hidden"], gnn_layers=cfg["gnn_layers"],
        lstm_hidden=cfg["lstm_hidden"], lstm_layers=cfg["lstm_layers"],
        dropout=cfg["dropout"], target_node=TARGET_NODE, n_classes=2
    ).to(DEVICE)

    # label extraction for pos_weight
    if stage_name == "trade":
        ytr = y_trade_arr[sample_t[idx_train]]
    elif stage_name == "dir":
        ytr = y_dir_arr[sample_t[idx_train]]
    else:
        raise ValueError("stage_name must be 'trade' or 'dir'")

    pos_w = make_pos_weight(ytr)
    loss_fn = nn.CrossEntropyLoss()  # базово CE
    # (если хочешь pos_weight, то лучше BCE; но чтобы не усложнять logits->2, оставим CE + баланс через sampler/веса)
    # Минимально: просто CE, а дисбаланс компенсируем тем, что Stage B обучается только на trade.

    opt = torch.optim.AdamW(model.parameters(), lr=cfg["lr"], weight_decay=cfg["weight_decay"])
    sch = torch.optim.lr_scheduler.ReduceLROnPlateau(opt, mode="max", factor=0.5, patience=4)
    scaler_amp = torch.amp.GradScaler('cuda', enabled=(cfg["use_amp"] and DEVICE.type=="cuda"))

    best_score = -1e9
    best_state = None
    patience = 8
    bad = 0

    hist = {"tr_loss":[], "va_loss":[], "va_f1m":[], "va_auc":[]}

    for ep in range(1, cfg["epochs"]+1):
        model.train()
        tot = 0.0; n = 0

        for x,e,y_trade_b,y_dir_b,er in tr_loader:
            x = x.to(DEVICE).float()
            e = e.to(DEVICE).float()

            # select labels by stage
            y = (y_trade_b if stage_name=="trade" else y_dir_b).to(DEVICE).long()

            opt.zero_grad(set_to_none=True)
            with torch.amp.autocast('cuda', enabled=(cfg["use_amp"] and DEVICE.type=="cuda")):
                logits = model(x, e, EDGE_INDEX.to(DEVICE))
                loss = loss_fn(logits, y)

            if not torch.isfinite(loss):
                continue

            scaler_amp.scale(loss).backward()
            scaler_amp.unscale_(opt)
            nn.utils.clip_grad_norm_(model.parameters(), cfg["grad_clip"])
            scaler_amp.step(opt)
            scaler_amp.update()

            tot += loss.item()*y.size(0)
            n += y.size(0)

        tr_loss = tot/max(n,1)

        # ---- eval (важно: eval_binary читает y из y_trade_b; поэтому подменяем порядок через wrapper-лоадер нельзя.
        # минимально: просто используем loader, но в eval_binary считаем y = y_trade_b.
        # => Для dir-stage нужно "перепаковать" y_dir в позицию y_trade_b (самый простой минимальный способ).
        def wrap_loader_for_stage(loader, stage):
            for x,e,y_trade_b,y_dir_b,er in loader:
                if stage == "trade":
                    yield x,e,y_trade_b,y_dir_b,er
                else:
                    # положить y_dir в slot y_trade_b
                    yield x,e,y_dir_b,y_dir_b,er

        va_loss, va_acc, va_f1m, va_auc, va_cm, va_y, va_prob, va_er = eval_binary(
            model, wrap_loader_for_stage(va_loader, stage_name), loss_fn
        )

        score = va_f1m  # минимально и стабильно
        sch.step(score)

        hist["tr_loss"].append(tr_loss)
        hist["va_loss"].append(va_loss)
        hist["va_f1m"].append(va_f1m)
        hist["va_auc"].append(va_auc)

        lr_now = opt.param_groups[0]["lr"]
        print(f"[{stage_name}] ep {ep:02d} lr={lr_now:.2e} tr={tr_loss:.4f} va={va_loss:.4f} f1m={va_f1m:.3f} auc={va_auc:.3f}")

        if score > best_score:
            best_score = score
            best_state = {k: v.detach().cpu().clone() for k,v in model.state_dict().items()}
            bad = 0
        else:
            bad += 1
            if bad >= patience:
                break

    if best_state is not None:
        model.load_state_dict(best_state)

    te_loss, te_acc, te_f1m, te_auc, te_cm, te_y, te_prob, te_er = eval_binary(
        model, wrap_loader_for_stage(te_loader, stage_name), loss_fn
    )

    res = {
        "best_val_score": best_score,
        "test_acc": te_acc,
        "test_f1m": te_f1m,
        "test_auc": te_auc,
        "test_cm": te_cm,
        "hist": hist,
        "test_y": te_y,
        "test_prob": te_prob,
        "test_er": te_er,
    }
    return model, res


## 9. Two-stage PnL by confidence thresholds

In [10]:
# ЛОГИЧЕСКИЙ БЛОК: PnL по порогам уверенности (two-stage)
# ИСПОЛНЕНИЕ БЛОКА:

def two_stage_pnl_by_threshold(
    prob_trade,          # (N,2) softmax: [:,1]=p_trade
    prob_dir,            # (N,2) softmax: [:,1]=p_up
    exit_ret,            # (N,) realized log-ret to TB exit
    thr_trade: float,
    thr_dir: float,
    cost_bps: float,
):
    p_trade = prob_trade[:,1]
    p_up = prob_dir[:,1]
    p_dn = 1.0 - p_up
    conf_dir = np.maximum(p_up, p_dn)

    trade_mask = (p_trade >= thr_trade) & (conf_dir >= thr_dir)

    action = np.zeros_like(exit_ret, dtype=np.float32)
    action[trade_mask] = np.where(p_up[trade_mask] >= 0.5, 1.0, -1.0)

    cost = (cost_bps * 1e-4) * trade_mask.astype(np.float32)
    pnl = action * exit_ret - cost

    out = {
        "n": len(exit_ret),
        "n_trades": int(trade_mask.sum()),
        "trade_rate": float(trade_mask.mean()),
        "pnl_sum": float(pnl.sum()),
        "pnl_mean": float(pnl.mean()),
        "pnl_sharpe": float((pnl.mean() / (pnl.std() + 1e-12)) * np.sqrt(288)),
    }
    return out

def sweep_thresholds(prob_trade, prob_dir, exit_ret, cfg):
    rows = []
    for thr_t in cfg["thr_trade_grid"]:
        for thr_d in cfg["thr_dir_grid"]:
            m = two_stage_pnl_by_threshold(
                prob_trade=prob_trade,
                prob_dir=prob_dir,
                exit_ret=exit_ret,
                thr_trade=thr_t,
                thr_dir=thr_d,
                cost_bps=cfg["cost_bps"],
            )
            rows.append({"thr_trade":thr_t, "thr_dir":thr_d, **m})
    return pd.DataFrame(rows).sort_values(["pnl_mean","pnl_sum"], ascending=False)

print("Two-stage PnL threshold utils ready.")


Two-stage PnL threshold utils ready.


## 10. Run folds: scale once → train trade → filter trades → train dir → PnL sweep

In [11]:
# ЛОГИЧЕСКИЙ БЛОК: run walk-forward folds for two-stage training
# ИСПОЛНЕНИЕ БЛОКА:

fold_rows = []
models_trade = []
models_dir = []

for fi, (idx_tr, idx_va, idx_te) in enumerate(walk_splits, 1):
    print("\n" + "="*70)
    print(f"FOLD {fi}/{len(walk_splits)} sizes:", len(idx_tr), len(idx_va), len(idx_te))

    # scale once per fold
    X_scaled, _ = fit_scale_nodes_train_only(X_node_raw, sample_t, idx_tr, max_abs=CFG["max_abs_feat"])

    # ---- Stage A: trade/no-trade on all samples
    m_trade, r_trade = train_binary_classifier(
        X_scaled, edge_feat,
        y_trade, y_dir,
        exit_ret, sample_t,
        idx_tr, idx_va, idx_te,
        CFG,
        stage_name="trade"
    )
    models_trade.append(m_trade)

    # ---- Stage B: direction ONLY on trade samples
    idx_tr_T = subset_trade_indices(idx_tr, sample_t, y_trade)
    idx_va_T = subset_trade_indices(idx_va, sample_t, y_trade)
    idx_te_T = subset_trade_indices(idx_te, sample_t, y_trade)

    # если в какой-то части trade почти нет — пропускаем fold direction
    if len(idx_tr_T) < max(200, CFG["batch_size"]*2) or len(idx_te_T) < 50:
        print("[dir] skip: not enough trade samples in this fold.")
        fold_rows.append({
            "fold": fi,
            "trade_test_f1m": r_trade["test_f1m"],
            "dir_test_f1m": np.nan,
            "best_pnl_mean": np.nan,
            "best_thr_trade": np.nan,
            "best_thr_dir": np.nan,
            "n_trades_best": np.nan,
        })
        continue

    m_dir, r_dir = train_binary_classifier(
        X_scaled, edge_feat,
        y_trade, y_dir,
        exit_ret, sample_t,
        idx_tr_T, idx_va_T, idx_te_T,
        CFG,
        stage_name="dir"
    )
    models_dir.append(m_dir)

    # ---- Two-stage PnL evaluation on TEST (в sample-space idx_te)
    # Получим probs trade на idx_te (из r_trade) и probs dir на idx_te тоже нужно.
    # r_dir тестился на idx_te_T, поэтому мы посчитаем prob_dir на idx_te через прогон модели по idx_te (без фильтра),
    # чтобы sweep был по всем точкам (trade-маска будет от модели trade).

    # helper: get probs on arbitrary indices
    @torch.no_grad()
    def predict_probs_on_indices(model, X_scaled, edge_feat, indices, stage_for_label="trade"):
        ds = LobGraphSequenceDataset2Stage(X_scaled, edge_feat, y_trade, y_dir, exit_ret, sample_t, indices, CFG["lookback"])
        loader = DataLoader(ds, batch_size=CFG["batch_size"], shuffle=False, collate_fn=collate_fn_2stage)
        model.eval()
        probs = []
        ers = []
        for x,e,yt,yd,er in loader:
            x = x.to(DEVICE).float()
            e = e.to(DEVICE).float()
            logits = model(x,e,EDGE_INDEX.to(DEVICE))
            p = torch.softmax(logits, dim=-1).cpu().numpy()
            probs.append(p)
            ers.append(er.cpu().numpy())
        return np.concatenate(probs), np.concatenate(ers)

    prob_trade_te, er_te = predict_probs_on_indices(m_trade, X_scaled, edge_feat, idx_te)
    prob_dir_te, _ = predict_probs_on_indices(m_dir, X_scaled, edge_feat, idx_te)

    # exit_ret on same points (already in er_te)
    sweep = sweep_thresholds(prob_trade_te, prob_dir_te, er_te, CFG)
    best = sweep.iloc[0].to_dict()

    print("PnL on test:",
          "| thr_trade=", best["thr_trade"],
          "| thr_dir=", best["thr_dir"],
          "| pnl_mean=", best["pnl_mean"],
          "| trades=", best["n_trades"])

    fold_rows.append({
        "fold": fi,
        "trade_test_f1m": r_trade["test_f1m"],
        "dir_test_f1m": r_dir["test_f1m"],
        "best_pnl_mean": best["pnl_mean"],
        "best_thr_trade": best["thr_trade"],
        "best_thr_dir": best["thr_dir"],
        "n_trades_best": best["n_trades"],
        "trade_rate_best": best["trade_rate"],
    })

summary = pd.DataFrame(fold_rows)
display(summary)
print("\nMEAN:")
print(summary.mean(numeric_only=True))



FOLD 1/4 sizes: 8386 1677 1677
[trade] ep 01 lr=2.00e-04 tr=0.6716 va=0.8684 f1m=0.348 auc=0.485
[trade] ep 02 lr=2.00e-04 tr=0.5983 va=0.8924 f1m=0.318 auc=0.473
[trade] ep 03 lr=2.00e-04 tr=0.5771 va=0.8433 f1m=0.396 auc=0.476
[trade] ep 04 lr=2.00e-04 tr=0.5722 va=0.9095 f1m=0.305 auc=0.476
[trade] ep 05 lr=2.00e-04 tr=0.5588 va=0.8786 f1m=0.369 auc=0.475
[trade] ep 06 lr=2.00e-04 tr=0.5492 va=0.9260 f1m=0.348 auc=0.472
[trade] ep 07 lr=2.00e-04 tr=0.5360 va=0.9298 f1m=0.378 auc=0.445
[trade] ep 08 lr=1.00e-04 tr=0.4997 va=1.1444 f1m=0.332 auc=0.425
[trade] ep 09 lr=1.00e-04 tr=0.4454 va=1.1094 f1m=0.390 auc=0.434
[trade] ep 10 lr=1.00e-04 tr=0.3912 va=1.1855 f1m=0.415 auc=0.465
[trade] ep 11 lr=1.00e-04 tr=0.3404 va=1.2387 f1m=0.434 auc=0.512
[trade] ep 12 lr=1.00e-04 tr=0.3049 va=1.4505 f1m=0.382 auc=0.526
[trade] ep 13 lr=1.00e-04 tr=0.2706 va=1.5379 f1m=0.406 auc=0.506
[trade] ep 14 lr=1.00e-04 tr=0.2549 va=1.6032 f1m=0.378 auc=0.500
[trade] ep 15 lr=1.00e-04 tr=0.2414 va=1.628

,fold,trade_test_f1m,dir_test_f1m,best_pnl_mean,best_thr_trade,best_thr_dir,n_trades_best,trade_rate_best
0,1,0.288304,0.444498,0.000172,0.5,0.50,642.0,0.382826
1,2,0.472549,0.399464,-0.000044,0.7,0.65,72.0,0.042934
2,3,0.358705,0.438351,-0.000528,0.7,0.70,599.0,0.357185
3,4,0.655167,0.380466,-0.000400,0.7,0.70,141.0,0.084079



MEAN:
fold                 2.500000
trade_test_f1m       0.443681
dir_test_f1m         0.415695
best_pnl_mean       -0.000200
best_thr_trade       0.650000
best_thr_dir         0.637500
n_trades_best      363.500000
trade_rate_best      0.216756
dtype: float64
